<a href="https://colab.research.google.com/github/kyungmo90/Ubion-Python/blob/main/%EC%9A%94%EC%96%91%EC%8B%9C%EC%84%A4_%EA%B0%84%EC%9D%98_%EC%88%98%EC%9D%98_%EA%B4%80%EA%B3%84%EC%97%90_%EB%94%B0%EB%A5%B8_%EC%8B%9C%EC%84%A4_%EC%88%98_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 요양시설 간의 수의 관계에 따른 시설 수 예측



인공신경망

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
#분류예측

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/2022년_9월_시군구별_장기요양기관_현황.CSV", encoding = "cp949")
df

,시도,노인요양시설,공동생활가정,방문요양,방문목욕,주야간보호,복지용구
0,서울특별시,5,3.0,92,71.0,17.0,18.0
1,서울특별시,9,13.0,111,102.0,19.0,20.0
2,서울특별시,8,23.0,96,81.0,22.0,9.0
3,서울특별시,20,14.0,185,144.0,31.0,30.0
4,서울특별시,10,19.0,120,103.0,26.0,13.0
...,...,...,...,...,...,...,...
245,경상남도,10,4.0,54,33.0,9.0,4.0
246,경상남도,9,NaN,25,19.0,12.0,NaN
247,경상남도,6,NaN,23,13.0,7.0,4.0
248,제주특별자치도,19,2.0,24,19.0,20.0,4.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도      250 non-null    object 
 1   노인요양시설  250 non-null    int64  
 2   공동생활가정  221 non-null    float64
 3   방문요양    250 non-null    int64  
 4   방문목욕    248 non-null    float64
 5   주야간보호   248 non-null    float64
 6   복지용구    239 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 13.8+ KB


In [14]:
df.isnull().sum()

시도         0
노인요양시설     0
공동생활가정    29
방문요양       0
방문목욕       2
주야간보호      2
복지용구      11
dtype: int64

In [18]:
df.fillna(0, inplace = True)

In [19]:
df.isnull().sum()

시도        0
노인요양시설    0
공동생활가정    0
방문요양      0
방문목욕      0
주야간보호     0
복지용구      0
dtype: int64

In [20]:
df.describe()

,노인요양시설,공동생활가정,방문요양,방문목욕,주야간보호,복지용구
count,250.000000,250.000000,250.00000,250.000000,250.000000,250.000000
mean,17.248000,7.172000,66.61600,46.684000,20.052000,7.892000
std,18.427174,8.699722,47.32631,36.347499,15.023185,6.740778
min,1.000000,0.000000,2.00000,0.000000,0.000000,0.000000
25%,6.000000,1.250000,29.00000,19.000000,9.000000,2.250000
50%,11.500000,4.000000,55.50000,35.000000,17.000000,6.000000
75%,21.000000,9.000000,92.00000,66.750000,27.750000,12.000000
max,129.000000,55.000000,236.00000,199.000000,88.000000,38.000000


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [22]:
df.columns

Index(['시도', '노인요양시설', '공동생활가정', '방문요양', '방문목욕', '주야간보호', '복지용구'], dtype='object')

In [27]:
import numpy as np

#분류예측

#1. 변수선택
X = df[['방문요양', '공동생활가정', '방문목욕', '주야간보호','복지용구']]
Y = df['노인요양시설']

#2. train = test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#3. 데이터 전처리(preprocessing)
ct = ColumnTransformer([("scaling", StandardScaler(), ['방문요양', '공동생활가정', '방문목욕', '주야간보호','복지용구'])])
ct.fit(X_train)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

In [30]:
#4. 모델링
from sklearn.neural_network import MLPRegressor

#4-1. 모델 생성
nn_reg_model = MLPRegressor(random_state=0, alpha=1, max_iter=1000, hidden_layer_sizes=[50, 50])
                            
#5. 모형학습 및 예측
nn_reg_model.fit(X_train, Y_train)
Y_pred = nn_reg_model.predict(X_test)

#6. 결과값 보고
print("Y predict value: \n", Y_pred)
print("trian accuracy : {:.3f}".format(nn_reg_model.score(X_train, Y_train)))
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(Y_test, Y_pred))
print("RMSE : {:3f}".format(rmse)) 

Y predict value: 
 [11.05293582  2.05857952  2.55038899 10.96007832 12.56363145 30.9769583
  3.81723982 13.89486604  2.00919218 15.33229728  1.33640401 17.00591281
 13.07725388  7.73360097 60.42769807  8.91677602 26.13985773 10.05946328
 25.02297741 12.99819609 15.95750522 70.3983231  13.59338917 11.62124593
 11.22751602  6.59245473 38.24964114 28.19127016  5.93264605  8.88154461
  4.65922535 41.69660593 46.33780754 38.04407498 22.0175843   4.50160792
  2.37032144 10.38409346  4.84061531  1.43685192  3.13613345  2.53618961
 24.99328335 20.49790453 13.06644658 26.16041961  5.69549199 28.84536463
 25.57697477  4.99804628 13.80886709 28.54507006  6.83254416 12.85146897
  9.54197241 37.40286319  9.97704772 22.23505636  8.91252759 13.1616445
  8.6352488  12.44315509 12.24209271  9.88396079 14.65445023  7.01988252
  7.56979776 42.29271426 32.25565876 10.79219201  5.37277389  9.77819124
  9.99219461 21.45254052 34.59076246]
trian accuracy : 0.848
RMSE : 14.115676


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


딥러닝

In [31]:
import warnings
warnings.filterwarnings("ignore")

DNN(Deep learning : Deep Neural Network)

In [63]:

#1. 변수선택
X = df[['방문요양', '공동생활가정', '방문목욕', '주야간보호','복지용구']]
Y = np.log1p(df['노인요양시설'])

#2. train = test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#3. 데이터 전처리(preprocessing)
ct = ColumnTransformer([("scaling", StandardScaler(), ['방문요양', '공동생활가정', '방문목욕', '주야간보호','복지용구'])])
ct.fit(X_train)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

In [50]:
Y.describe()

count    250.000000
mean      17.248000
std       18.427174
min        1.000000
25%        6.000000
50%       11.500000
75%       21.000000
max      129.000000
Name: 노인요양시설, dtype: float64

In [51]:
X_train[0]

array([ 0.76515483,  0.62857334,  0.81137684,  0.73132306, -0.31745607])

In [66]:
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.metrics import Accuracy

#5. 시드값 설정 : 시작점
np.random.seed(0)
tf.random.set_seed(0)

#6. 모형생성
model = keras.models.Sequential()
model.add(keras.layers.Dense(12, input_dim=5, activation = "relu"))


#7. 모형 학습
model.compile(loss = "mse", optimizer = "SGD")
model.fit(X_train, Y_train, validation_split=0.2, epochs = 100, batch_size = 64, verbose = 2)


Epoch 1/100
3/3 - 1s - loss: nan - val_loss: nan - 528ms/epoch - 176ms/step
Epoch 2/100
3/3 - 0s - loss: nan - val_loss: nan - 28ms/epoch - 9ms/step
Epoch 3/100
3/3 - 0s - loss: nan - val_loss: nan - 34ms/epoch - 11ms/step
Epoch 4/100
3/3 - 0s - loss: nan - val_loss: nan - 31ms/epoch - 10ms/step
Epoch 5/100
3/3 - 0s - loss: nan - val_loss: nan - 47ms/epoch - 16ms/step
Epoch 6/100
3/3 - 0s - loss: nan - val_loss: nan - 47ms/epoch - 16ms/step
Epoch 7/100
3/3 - 0s - loss: nan - val_loss: nan - 26ms/epoch - 9ms/step
Epoch 8/100
3/3 - 0s - loss: nan - val_loss: nan - 28ms/epoch - 9ms/step
Epoch 9/100
3/3 - 0s - loss: nan - val_loss: nan - 26ms/epoch - 9ms/step
Epoch 10/100
3/3 - 0s - loss: nan - val_loss: nan - 29ms/epoch - 10ms/step
Epoch 11/100
3/3 - 0s - loss: nan - val_loss: nan - 28ms/epoch - 9ms/step
Epoch 12/100
3/3 - 0s - loss: nan - val_loss: nan - 29ms/epoch - 10ms/step
Epoch 13/100
3/3 - 0s - loss: nan - val_loss: nan - 47ms/epoch - 16ms/step
Epoch 14/100
3/3 - 0s - loss: nan - v